# Creating housing archetypes using K-means method
### Q4 2020-21

## Introduction
This workbook will ...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from kneed import KneeLocator 
from sklearn.datasets import make_blobs 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import ParameterGrid
from itertools import permutations
from itertools import combinations

## Data Source
The data represents a subset of ERS records used for initial analysis.

In [ ]:
ers_sample_records = pd.read_csv(r"C:\Users\owner\Documents\NRCan\code\practice\InitialHousingData.csv",)
ers_sample_records 

#original amount of obsevations/rows 
og_obs = ers_sample_records.shape[0]

## Database Preperation and Varibable Selection 


### Select variables for clustering
Placeholder for this process. 

In [ ]:
# Air50P selected for 1D clustering test
cl_variables = ['Air50P']
test_data = ers_sample_records[cl_variables]


### Missing values

In [ ]:
#Remove rows with blank values
test_data_cleaned = test_data.dropna()
#display how many rows removed

###  inconsistent data removal 
each variable has own range that is should be within. Airt tightness >0 ??


### Ensure all values are numerical
and positive? 

In [ ]:
# if there is catagorical information use LabelEncoder,onehotenoder from scikitlearn or ... creat if str then, if not then float

# ensure all vales are floats 
test_data_cleaned.astype('float') #float64?

### Remove outliers

Visualize with box plot to confirm outlier removal.

In [ ]:
#box plot before outlier removal
test_data_cleaned.boxplot() 

#### Local outlier factor

In [ ]:
#define the model

lof = LocalOutlierFactor()
lof_pred = lof.fit_predict(test_data_cleaned) 

#We'll extract the negative outputs as the outliers.
mask = lof_pred != -1

#remove rows with outliers 
lof_data = test_data_cleaned[mask] #i dont think this works for sets with more than 1 varibale
print([lof_data])

#print amount of points deleated

outliers_rem = og_obs - lof_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

#plot without after removal

lof_data.boxplot()

#### Z-score 

In [ ]:
#find absolute value of z-score for each observation
z = np.abs(stats.zscore(test_data_cleaned))

#only keep rows in dataframe with all z-scores less than absolute value of 3 
z_data = test_data_cleaned[(z<3).all(axis=1)]

#find how many rows are left in the dataframe 
z_data

outliers_rem = og_obs - z_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

In [ ]:
#plot 
z_data.boxplot()

#### IRQ

In [ ]:
#find Q1, Q3, and interquartile range for each column
Q1 =test_data_cleaned.quantile(q=.25)
Q3 = test_data_cleaned.quantile(q=.75)
IQR =test_data_cleaned.apply(stats.iqr)

#only keep rows in dataframe that have values within 1.5*IQR of Q1 and Q3
iqr_data = test_data_cleaned[~((test_data_cleaned < (Q1-1.5*IQR)) | (test_data_cleaned > (Q3+1.5*IQR))).any(axis=1)]

#find how many rows are left in the dataframe 

outliers_rem = og_obs - iqr_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

In [ ]:
#plot 
iqr_data.boxplot()


 Mahalanobis distance - calc when multi varibale later
 https://www.statology.org/mahalanobis-distance-python/

### clusterss stuff

In [ ]:
# create sets of pre-processed data ready for clustering 
scaled_data_sets = [iqr_data,lof_data, z_data]


In [ ]:
#list of scalers
standard = StandardScaler()
minimax = MinMaxScaler()
scalers = [standard, minimax]
#list of initalizers 
r = 'random'
plus = 'k-means++' 
initalizer = [r, plus]



#need to figure out how to use paramgrid to do this
#permutatins
perm_amount = len(scalers)*len(initalizer)*len(scaled_data_sets)

i=0 # data set
j=0 #scaler
l=0 #init
m=0 #rounds 
while i != len(scaled_data_sets):
     #first set of data to scale
    scaled_features = scalers[j].fit_transform(scaled_data_sets[i]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":initalizer[l],  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init=initalizer[l],
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    print(kmeans.inertia_) # can use other validations too 
    
    
    
   

            
    
    

In [ ]:
# datasets being transfromed with standard scaler and random init
standard_rand=[]
for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[0].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"random",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="random",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    standard_rand.append(kmeans.inertia_) # can use other validations too 
    

In [ ]:
#datasets being transfromed with mini scaler and random init
mini_rand=[]
for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[1].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"random",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="random",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    mini_rand.append(kmeans.inertia_) # can use other validations too 

In [ ]:
#datasets being transfromed with mini scaler and k++ init
mini_plus=[]
for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[1].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"k-means++",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    mini_plus.append(kmeans.inertia_) # can use other validations too 

In [ ]:
#datasets being transfromed with standard scaler and k++ init
standard_plus=[]
for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[0].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"k-means++",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    standard_plus.append(kmeans.inertia_) # can use other validations too 

In [ ]:

results = pd.DataFrame({ 'st rand':standard_rand, 'st plus':standard_plus,'min rand':mini_rand, 'min plus': mini_plus}, index= ['IRQ','LOF','Z-score'])
results


In [ ]:
#choose best scheme based on interia 
results.iloc[1,0] #LOF, Standard, Plus

### Clusteting with determined parameters

Take parameters from above and re-cluster

Cluster centers

In [ ]:
#kmeans.cluster_centers_

Iterations

In [ ]:
#kmeans.n_iter_

#### Resutling Clusters

In [ ]:
#cl_labels=kmeans.labels_ #create variale for lables

 add cluster labes as a column 

In [ ]:
#cl_data.plot.scatter(x='Air50P',y='clusterNo')#plot clusters. when multi-dim need to figure out how to colour points by cluster


In [ ]:
#cl_data.value_counts('clusterNo') #number of houses in each cluster 

### Silhouette Coefficicent 
another clustering amount determination method

In [ ]:
silhouette_coefficients =[]
for k in range(2,11):
    kmeans=KMeans(n_clusters=k,**kmeans_kwargs)
    kmeans.fit(scaled_features)
    score = silhouette_score(scaled_features, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()